In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [49]:
# data = pd.read_csv('training_dataset/raw_dataset.csv')
# data = pd.read_csv('training_dataset/raw_scores_pvallog.csv')
# data = pd.read_csv('training_dataset/ionocyte_raw_dataset.csv')
data = pd.read_csv('training_dataset/ionocyte_scores_pvallog.csv')

In [50]:
# data = pd.read_csv('ionocyte_raw_dataset.csv')
data['disease_ontology_label'] = (data['disease_ontology_label'] == 'COVID-19').astype(int)
X = data.drop(['NAME', 'disease_ontology_label'], axis=1)
y = data['disease_ontology_label']

# print("Class sizes:")
# print(y.value_counts())

In [51]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# # Impute missing values
# imputer = SimpleImputer(strategy='median')
# X_train = imputer.fit_transform(X_train)
# X_test = imputer.transform(X_test)


# # Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# # Feature selection using SelectKBest with f_classif
selector = SelectKBest(score_func=f_classif, k=50)  # Select top 100 features
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)


# Display the number of features after feature selection
print(f"Total number of features after selection: {X_train.shape[1]}")

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Increased from 64 to 256 units
    Dropout(0.3),  # Increased dropout rate to handle more complex model
    Dense(256, activation='relu'),  # Added an additional Dense layer
    Dropout(0.2),  # Maintain dropout to prevent overfitting
    Dense(64, activation='relu'),  # Further layer to increase model depth
    Dropout(0.2),  # Consistent dropout rate for complexity
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=40, batch_size=10, validation_split=0.2, verbose=1)

Total number of features after selection: 50
Epoch 1/40


c:\Users\atif7\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 322  864 1038 1568 1765 1766 1767 1768 1772 1775 1776 1777 1778 1779
 1780 1781 1782 1785 1786 1788 1789 1790 1791 1792 1796 1798 1800 1801
 1804 1805 1806 1807 1810 1812 1813 1815 1816 1817 1818 1819 1822 1823
 1824 1826 1828 1832 1833 1835 1836 1837 1838 1841 1842 1843 1844 1849
 1851 1852 1853 1856 1857 1860 1861 1863 1868 1869 1870 1876 1877 1878
 1879 1880 1881 1882 1883 1884 1885 1888 1889 1890 1891 1892 1894 1895
 1896 1898 1901 1903 1905 1906 1907 1909 1910 1911 1912 1913 1914 1915
 1916 1918 1919 1920 1921 1922 1923 1924 1927 1929 1930 1931 1933 1934
 1937 1940 1942 1943 1944 1948 1951 1953 1956 1959 1962 1963 1966 1968
 1969 1972 1973 1978 1980 1982 1983 1984 1985 1986 1990 1991 1994 1996
 1997 1999 2000 2001 2003 2005 2006 2011 2012 2013 2014 2015 2017 2019
 2022 2023 2026 2028 2029 2030 2032 2034 2036 2039 2040 2043 2044 20

38/38 [==============================] - 2s 9ms/step - loss: 0.6148 - accuracy: 0.6496 - val_loss: 0.4814 - val_accuracy: 0.7849
Epoch 2/40
38/38 [==============================] - 0s 4ms/step - loss: 0.4727 - accuracy: 0.7736 - val_loss: 0.4466 - val_accuracy: 0.8172
Epoch 3/40
38/38 [==============================] - 0s 4ms/step - loss: 0.4389 - accuracy: 0.7871 - val_loss: 0.4582 - val_accuracy: 0.8065
Epoch 4/40
38/38 [==============================] - 0s 3ms/step - loss: 0.4168 - accuracy: 0.8113 - val_loss: 0.4522 - val_accuracy: 0.7957
Epoch 5/40
38/38 [==============================] - 0s 4ms/step - loss: 0.4054 - accuracy: 0.8275 - val_loss: 0.4374 - val_accuracy: 0.8172
Epoch 6/40
38/38 [==============================] - 0s 3ms/step - loss: 0.3909 - accuracy: 0.8032 - val_loss: 0.4439 - val_accuracy: 0.8172
Epoch 7/40
38/38 [==============================] - 0s 3ms/step - loss: 0.3298 - accuracy: 0.8464 - val_loss: 0.4473 - val_accuracy: 0.7957
Epoch 8/40
38/38 [=============

In [52]:
# Evaluate the model on the test data
results = model.evaluate(X_test, y_test)
print("Test Loss, Test Accuracy:", results)

4/4 [==============================] - 0s 3ms/step - loss: 1.7868 - accuracy: 0.7521
Test Loss, Test Accuracy: [1.786791205406189, 0.752136766910553]
